In [1]:
from cdapython import Q
import json

### Image files for a single subject

In [2]:

q1 = Q('id = "TCGA-13-1504"')
r = q1.run()
print(r)


Getting results from database

Total execution time: 9338 ms

QueryID: af1a129e-e481-4edc-bf37-9fdc85561a0b
Query: SELECT all_v2.* FROM gdc-bq-sample.integration.all_v2 AS all_v2 WHERE (all_v2.id = 'TCGA-13-1504')
Offset: 0
Count: 1
Total Row Count: 1
More pages: False



In [3]:
from collections import Counter
categories = Counter()
imaging_types = Counter()
for f in r[0]['File']:
    dc = f['data_category']
    categories[dc] += 1
    if f['data_category'] == "Imaging":
        imaging_types[f['data_type']+'/'+f['file_format']] += 1
print(categories)
print("\nImaging type/format")
print(imaging_types)

Counter({'Imaging': 226, 'Peptide Spectral Matches': 55, 'Raw Mass Spectra': 28, 'Processed Mass Spectra': 27, 'Simple Nucleotide Variation': 16, 'Biospecimen': 12, 'Clinical': 8, 'Copy Number Variation': 7, 'Sequencing Reads': 3, 'Transcriptome Profiling': 2, 'DNA Methylation': 1, 'Proteome Profiling': 1})

Imaging type/format
Counter({'CT/DICOM': 226})


### Use DRS to get some details of each file
For illustration here's what DRS can tell us about one Imaging file

In [4]:
for f in r[0]['File'] :
    if f['data_category'] == "Imaging":
        drs_id = f['id']
        print(drs_id)
        break

f5027f67-011b-4f8e-a9bd-de73eaaf2fc0


In [5]:
from fasp.loc import crdcDRSClient
drsClient = crdcDRSClient()
drsClient.getObject(drs_id)

{'access_methods': [{'access_id': 'gs',
   'access_url': {'url': 'gs://idc-tcia-tcga-ov/dicom/1.3.6.1.4.1.14519.5.2.1.9203.4007.107225085018428492152614916489/1.3.6.1.4.1.14519.5.2.1.9203.4007.133576245722946348511732255420/1.3.6.1.4.1.14519.5.2.1.9203.4007.472644011742805252569503604125.dcm#1592638563565092'},
   'region': '',
   'type': 'gs'}],
 'aliases': [],
 'checksums': [{'checksum': '51f38e0cd236104a0a88a1a88b371155',
   'type': 'md5'}],
 'contents': [],
 'created_time': '2020-09-18T20:21:07.664086',
 'description': None,
 'form': 'object',
 'id': 'dg.4DFC/f5027f67-011b-4f8e-a9bd-de73eaaf2fc0',
 'mime_type': 'application/json',
 'name': None,
 'self_uri': 'drs://nci-crdc.datacommons.io/dg.4DFC/f5027f67-011b-4f8e-a9bd-de73eaaf2fc0',
 'size': 527958,
 'updated_time': '2020-09-18T20:21:07.664091',
 'version': 'e2be4764'}

To someone with some knowledge of dicom the access url field gives us a clue. This looks like a path to  nested directories named with dicom instance ids. The nesting of those directories reflects the dicom data model. That data model gives meaning to the files which is otherwise absent. In fact the full meaning is only within the files.

We can also see that there is a specific GCS bucket for the TCGA OV study (spoiler: study is overloaded, as we shall shortly discover :-)

Note the use of the word 'clue'. This is indeed detective work. Users should not have to do detective work.

In [6]:
import json
nn = 1
nmax = 1000
drs_list = []
for f in r[0]['File'] :
    if f['data_category'] == "Imaging":
        drs_id= 'crdc:'+f['id']
        print(drs_id)
        drs_list.append(mr.getObject(drs_id))
        nn += 1
    if nn > nmax:
        break
with open('drs_list.json', 'w') as outfile:
    json.dump(drs_list, outfile)

crdc:f5027f67-011b-4f8e-a9bd-de73eaaf2fc0


NameError: name 'mr' is not defined

Write a tab delimited file containing the elements of the file path. i.e. extract the DICOM structure. If the path does not fit the pattern of 7 elements list it out.

In [ ]:
import pandas as pd
dicom_lst = []
with open('dicom_list2.txt', 'w') as outfile:
    for d in drs_list:
        url = d['access_methods'][0]['access_url']['url']
        parts = url.split("/")
        if len(parts) == 7: 
            dicom_lst.append(parts[2:7])
        else:
            print(parts)
            print(json.dumps(d, indent=3))
        outstr = '\t'.join(parts)
        outfile.write(outstr+'\n')
        
df = pd.DataFrame(dicom_lst, columns =['bucket', 'folder', 'study', 'series', 'instance'])
df

How many studies?

In [ ]:
df.groupby('study').size()

Just one study. So what series does it have within it, and how many instances (images) within each series?

In [ ]:
df.groupby('series').size()

There are 4 series each with widely varying numbers of instances within them. What a dicom instance is varies according to the imaging modality. The most common case is that an instance is a image.

What each series is is not clear from the ids above (btw these ids are dicom instance ids).

Note that we had to go through detective work to do this.


In [ ]:
import requests
import os
def download(url, file_path):
    with open(os.path.expanduser(file_path), "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [ ]:
drs_id = 'crdc:72eacf56-f87d-4af4-8128-47a6897d1506'
drs_url = mr.getAccessURL(drs_id, 'gs')
download(drs_url, '/Users/forei/Downloads/72eacf56-f87d-4af4-8128-47a6897d1506.dcm')

In [ ]:
mr.getObject(drs_id)

What if we do this across the whole study?


In [ ]:
q1 = Q('id = "TCGA-13-1504"')
r = q1.run()
print(r)

### TCGA-06-0118
Have previously downloaded dicom files from NBIA for this patient.

In [ ]:
q1 = Q('subject_associated_project = "TCGA-OV"')
r = q1.run()
print(r)

In [ ]:
q1 = Q('ResearchSubject.associated_project = "TCGA-OV"')
r = q1.run()
print(r)


In [ ]:
query1 = """SELECT su.id sid , rs.identifier rsid, fi.drs_uri
from gdc-bq-sample.integration.all_v2 AS su,
unnest(ResearchSubject) AS rs,
unnest(rs.File) as fi
where (su.id = 'TCGA-13-1504')
and file_format = 'BAM' """

r1 = Q.sql(query1)
r1

In [ ]:
for res in r1:
    print(json.dumps(res, indent=3))

In [ ]:
query2 = """SELECT su.id sid , rs.identifier rsid, fi.*
from gdc-bq-sample.integration.all_v2 AS su,
unnest(ResearchSubject) AS rs,
unnest(rs.File) as fi
where (su.id = 'TCGA-13-1504')
and file_format = 'BAM' """

r2 = Q.sql(query2)
r2

In [ ]:
for res in r2:
    print(json.dumps(res, indent=3))